In [1]:
import pandas as pd

# Đọc file ticket
df_ticket = pd.read_excel("C:\\Users\\Nguyen_Thanh_Vinh\\Downloads\\processed_ticket_cleaned.xlsx", sheet_name = "TICKET")

customer = pd.read_excel("C:\\Users\\Nguyen_Thanh_Vinh\\Downloads\\processed_ticket_cleaned.xlsx", sheet_name = "CUSTOMER")

# Đọc file listedin
df_listed = pd.read_excel("C:\\Users\\Nguyen_Thanh_Vinh\\Downloads\\MOVIE _ TABLE.xlsx", sheet_name = "LISTEDIN")

In [ ]:
# Với những order có saledate là null thì dùng date (ngày xem) thay thế
df_ticket['saledate'] = df_ticket['saledate'].fillna(df_ticket['date'])

In [3]:
# Đảm bảo ngày ở đúng định dạng
df_ticket['saledate'] = pd.to_datetime(df_ticket['saledate'])

# Lấy ngày phân tích là ngày mới nhất
analysis_date = df_ticket['saledate'].max()

# Bỏ trùng theo orderid để không bị đếm dư tiền
unique_orders = df_ticket.drop_duplicates(subset='orderid')

# Tính RFM
rfm = unique_orders.groupby('customerid').agg({
    'saledate': lambda x: (analysis_date - x.max()).days,  # Recency: từ lần mua mới nhất
    'orderid': 'count',                                    # Frequency: số đơn hàng
    'total': 'sum'                                         # Monetary: tổng số tiền đã chi
}).reset_index()

# Đổi tên cột
rfm.columns = ['customerid', 'Recency', 'Frequency', 'Monetary']

# Hiển thị thử
print(rfm.head())

   customerid  Recency  Frequency  Monetary
0  0000000014       25          1     90000
1  0000000034        1          2    180000
2  0000000051        9          1     90000
3  0000000081       26          2    180000
4  0000000098        6          2    180000


In [4]:
# Chuẩn hóa tên phim
df_ticket["film"] = df_ticket["film"].str.strip().str.lower()
df_listed["title"] = df_listed["title"].str.strip().str.lower()

# Merge
df_merged = pd.merge(df_ticket, df_listed, left_on="film", right_on="title", how="left")

In [5]:
df_merged

,orderid,cashier,saledate,customerid,date,time,room,film,ticketcode,slot,...,languages2,release_year,rating_viet,duration,listed_in_1,listed_in_2,listed_in_3,listed_in_4,listed_in_5,listed_in_6
0,10006052019B0225,emp002,2019-05-06 16:40:43,0000000014,2019-05-06,18:15:00,3,avengers: hồi kết,20074925,G04,...,NaN,2019,C13,182,Action-Adventure,Fantasy,Science Fiction,NaN,NaN,NaN
1,10006052019B0225,emp002,2019-05-06 16:40:43,0000000014,2019-05-06,18:15:00,3,avengers: hồi kết,20074924,G03,...,NaN,2019,C13,182,Action-Adventure,Fantasy,Science Fiction,NaN,NaN,NaN
2,10022052019B0167,emp011,2019-05-22 20:30:28,0000000034,2019-05-22,20:45:00,2,john wick 3: chuẩn bị chiến tranh (c18),20097042,E04,...,NaN,2019,C18,127,Action,Crime,Thriller,NaN,NaN,NaN
3,10022052019B0167,emp011,2019-05-22 20:30:28,0000000034,2019-05-22,20:45:00,2,john wick 3: chuẩn bị chiến tranh (c18),20097041,E03,...,NaN,2019,C18,127,Action,Crime,Thriller,NaN,NaN,NaN
4,10130052019B0141,emp005,2019-05-30 19:28:44,0000000034,2019-05-30,19:30:00,1,ngôi đền kỳ quái (c18),20106761,F04,...,NaN,2019,C18,108,Comedy,Horror,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35503,10111052019B0123,emp002,2019-05-11 14:26:45,KH6166700,2019-05-11,14:10:00,4,mẹ ma than khóc la llorona (c18),44068,F3,...,Spanish,2019,C18,98,Horror,Mystery,Thriller,NaN,NaN,NaN
35504,10111052019B0123,emp002,2019-05-11 14:26:45,KH6166700,2019-05-11,14:10:00,4,mẹ ma than khóc la llorona (c18),69208,D17,...,Spanish,2019,C18,98,Horror,Mystery,Thriller,NaN,NaN,NaN
35505,10111052019B0123,emp002,2019-05-11 14:26:45,KH6166700,2019-05-11,14:10:00,4,mẹ ma than khóc la llorona (c18),21512,F2,...,Spanish,2019,C18,98,Horror,Mystery,Thriller,NaN,NaN,NaN
35506,10111052019B0123,emp002,2019-05-11 14:26:45,KH6166700,2019-05-11,14:10:00,4,mẹ ma than khóc la llorona (c18),45054,B1,...,Spanish,2019,C18,98,Horror,Mystery,Thriller,NaN,NaN,NaN


In [6]:
# # Tổng tiền mỗi khách hàng
# df_total = df_merged.groupby("customerid")["total"].sum().reset_index()
# df_total.rename(columns={"total": "total_price"}, inplace=True)

In [7]:
# Gộp tất cả cột thể loại lại thành một danh sách
genre_cols = ['listed_in_1', 'listed_in_2', 'listed_in_3', 'listed_in_4', 'listed_in_5', 'listed_in_6']

# Chuẩn hóa và loại bỏ thể loại trùng trong mỗi vé
def clean_genres(row):
    genres = [g for g in row if pd.notna(g)]
    # chuyển về lowercase và loại khoảng trắng
    genres = [g.strip().lower() for g in genres]
    
    # ánh xạ các thể loại tương đương về 1 nhóm
    mapping = {
        'sci-fi': 'sci-fi',
        'sci fi': 'sci-fi',
        'science fiction': 'sci-fi',
        'action': 'action',
        'action-adventure': 'action-adventure',
        'animation': 'animation',
        'comedy': 'comedy',
        'crime': 'crime',
        'drama': 'drama',
        'family': 'family',
        'fantasy': 'fantasy',
        'horror': 'horror',
        'melodrama': 'melodrama',
        'mystery': 'mystery',
        'romance': 'romance',
        'thriller': 'thriller'
    }
    # map và loại trùng
    genres = list(set([mapping.get(g, g) for g in genres]))
    return genres

df_merged['all_genres'] = df_merged[genre_cols].apply(clean_genres, axis=1)

In [8]:
# Tạo bảng explode (mỗi thể loại là một dòng)
genre_exploded = df_merged[['customerid', 'all_genres']].explode('all_genres')

# Đếm số lần từng khách hàng xem từng thể loại
genre_count = genre_exploded.groupby(['customerid', 'all_genres']).size().unstack(fill_value=0).reset_index()

In [9]:
# Chuyển 'time' thành kiểu datetime.time
df_ticket["time"] = pd.to_datetime(df_ticket["time"], format="%H:%M:%S").dt.time

# Hàm phân loại khung giờ
def get_time_slot(t):
    h = t.hour
    if 5 <= h <= 10:
        return "Sáng"
    elif 11 <= h <= 13:
        return "Trưa"
    elif 14 <= h <= 17:
        return "Chiều"
    elif 18 <= h <= 21:
        return "Tối"
    else:
        return "Khuya"

df_ticket["time_slot"] = df_ticket["time"].apply(get_time_slot)

# Đếm số vé mỗi khách mua trong từng khung giờ
df_time_count = df_ticket.groupby(["customerid", "time_slot"]).size().unstack(fill_value=0).reset_index()

In [10]:
df_ticket['zone'] = df_ticket['slot'].astype(str).str[0]

df_zone_count = df_ticket.groupby(['customerid', 'zone']).size().unstack(fill_value=0).reset_index()

In [11]:
df_customer = pd.merge(rfm, genre_count, on="customerid", how="left")
df_customer = pd.merge(df_customer, df_time_count, on="customerid", how="left")
df_customer = pd.merge(df_customer, df_zone_count, on="customerid", how="left")

In [12]:
df_customer

,customerid,Recency,Frequency,Monetary,action,action-adventure,animation,comedy,crime,drama,...,B,C,D,E,F,G,H,I,J,K
0,0000000014,25,1,90000,0,2,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
1,0000000034,1,2,180000,2,0,0,2,2,0,...,0,0,0,2,2,0,0,0,0,0
2,0000000051,9,1,90000,2,0,0,0,2,0,...,0,0,0,2,0,0,0,0,0,0
3,0000000081,26,2,180000,0,4,0,0,0,0,...,0,0,0,2,2,0,0,0,0,0
4,0000000098,6,2,180000,0,2,0,2,2,0,...,0,0,0,2,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4474,KH9855766,0,3,270000,2,3,0,0,2,0,...,0,0,0,0,0,0,0,2,3,0
4475,KH9958204,6,1,135000,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,3
4476,WEBS00000043900,22,2,180000,0,0,0,2,0,0,...,0,0,0,0,2,0,1,0,0,0
4477,WEBS00000044909,1,1,180000,0,4,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0


In [13]:
df_customer = pd.merge(df_customer, customer, on="customerid", how="left")

In [14]:
df_customer

,customerid,Recency,Frequency,Monetary,action,action-adventure,animation,comedy,crime,drama,...,H,I,J,K,DOB,gender,address,Website,job,industry
0,0000000014,25,1,90000,0,2,0,0,0,0,...,0,0,0,0,36268,Nữ,hoa khe quan thanh khe,KH0104|0345,student,computer
1,0000000034,1,2,180000,2,0,0,2,2,0,...,0,0,0,0,35901,Nam,man thai,KH0104|0333,student,health service
2,0000000051,9,1,90000,2,0,0,0,2,0,...,0,0,0,0,34319,Nam,que son quang nam,KH0104|0255,blue collar,economics
3,0000000081,26,2,180000,0,4,0,0,0,0,...,0,0,0,0,36472,Nam,truong chinh,KH0104|0293,student,economics
4,0000000098,6,2,180000,0,2,0,2,2,0,...,2,0,0,0,34608,Nam,cẩm lệ,KH0104|40580,blue collar,health service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4474,KH9855766,0,3,270000,2,3,0,0,2,0,...,0,2,3,0,34885,Nữ,"A, THUAN PHUOC,HAI CHAU, DA NANG",NaN,specialist,finance
4475,KH9958204,6,1,135000,0,0,0,3,0,0,...,0,0,0,3,36892,Nữ,"Phước Mỹ, ĐN",NaN,teenager,NaN
4476,WEBS00000043900,22,2,180000,0,0,0,2,0,0,...,1,0,0,0,32311,Nữ,nguyen gian thanh,KHWEBS|00006890,specialist,construction
4477,WEBS00000044909,1,1,180000,0,4,0,0,0,0,...,0,0,0,0,35316,Nữ,ngũ hành sơn,KHWEBS|00007899,blue collar,finance


In [15]:
df_customer.to_excel("C:\\Users\\Nguyen_Thanh_Vinh\\Downloads\\CUSTOMER SEGMENTATION.xlsx", index = None)